In [20]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import yaml

In [49]:
load_dotenv('.env')
api_key = os.environ.get("AV_API_KEY")

In [23]:
# Load config data
with open(f"../config.yaml") as stream:
    config = yaml.safe_load(stream)

In [24]:
# Variables: Note: these are case-sensitive
# from_symbol = 'USD'
to_symbol = 'RUB'

In [25]:
base_url = f'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol={to_symbol}&outputsize=full&apikey={api_key}'
r = requests.get(base_url)
response_data = r.json()
response_data

{'Meta Data': {'1. Information': 'Forex Daily Prices (open, high, low, close)',
  '2. From Symbol': 'USD',
  '3. To Symbol': 'RUB',
  '4. Output Size': 'Full size',
  '5. Last Refreshed': '2022-10-10 10:10:00',
  '6. Time Zone': 'UTC'},
 'Time Series FX (Daily)': {'2022-10-10': {'1. open': '62.15000',
   '2. high': '63.40000',
   '3. low': '60.87500',
   '4. close': '62.90200'},
  '2022-10-09': {'1. open': '62.15000',
   '2. high': '62.15000',
   '3. low': '62.15000',
   '4. close': '62.15000'},
  '2022-10-06': {'1. open': '60.49000',
   '2. high': '62.56050',
   '3. low': '60.49000',
   '4. close': '60.49000'},
  '2022-10-04': {'1. open': '58.99000',
   '2. high': '60.99000',
   '3. low': '58.09000',
   '4. close': '58.99000'},
  '2022-10-03': {'1. open': '57.99010',
   '2. high': '59.46200',
   '3. low': '57.99000',
   '4. close': '57.99000'},
  '2022-10-02': {'1. open': '60.21500',
   '2. high': '60.21500',
   '3. low': '57.19000',
   '4. close': '60.21500'},
  '2022-09-29': {'1. op

In [26]:
def extract_fx_rates(
    to_symbol:str,
    api_key:str
)->pd.DataFrame:
    """
    Extracts daily exchange rates.
    - `from_symbol`: The currency symbol from which the exchange occurs.
    - `to_symbol`: The currency symbol into which the exchange occurs.
    
    """
    base_url = f'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol={to_symbol}&outputsize=full&apikey={api_key}'
    r = requests.get(base_url)
    if r.status_code == 200:
        response_data = r.json()
        df = pd.DataFrame(response_data['Time Series FX (Daily)']).transpose().reset_index()
        df = df.rename(columns={ df.columns[0]: "Date" })
        # df['from'] = f'{from_symbol}' # Won't use in case we do only USD in the FROM column
        df['to'] = f'{to_symbol}'
        return df

In [27]:
df = extract_fx_rates(to_symbol=to_symbol, api_key=api_key)
dfdf_currencies = config['extract']['exchange_rate']['currency']

,Date,1. open,2. high,3. low,4. close,to
0,2022-10-10,62.15000,63.40000,60.87500,62.90200,RUB
1,2022-10-09,62.15000,62.15000,62.15000,62.15000,RUB
2,2022-10-06,60.49000,62.56050,60.49000,60.49000,RUB
3,2022-10-04,58.99000,60.99000,58.09000,58.99000,RUB
4,2022-10-03,57.99010,59.46200,57.99000,57.99000,RUB
...,...,...,...,...,...,...
2600,2012-10-15,30.96320,30.99100,30.82560,30.96500,RUB
2601,2012-10-14,31.06030,31.26420,30.95200,31.10620,RUB
2602,2012-10-11,31.03400,31.22200,30.93100,30.96840,RUB
2603,2012-10-10,31.11510,31.21700,30.96600,31.11880,RUB


In [31]:
def extract():
    df_currencies = config['extract']['exchange_rate']['currency']
    df_concat = pd.DataFrame()
    for currency_name in df_currencies:
        df_extracted = extract_fx_rates(to_symbol=currency_name, api_key=api_key)
        df_concat = pd.concat([df_concat, df_extracted])
    return df_concat.reset_index().drop(labels=["index"], axis=1)

In [21]:
# Trying to figure out: how to use parameter as df name to create df per exchange currency?
# (Unrecommended?) way of doing this: https://stackoverflow.com/questions/67117157/is-there-a-way-to-made-the-name-of-a-dataframe-a-variable-that-is-defined-by-the
# def extract():
#     df_currencies = pd.read_csv("data/main_currencies.csv")
#     df_concat = pd.DataFrame()
#     for val in df_currencies["currency code"]:
#         globals()["table_"f"{val}"] = extract_fx_rates(from_symbol=from_symbol, to_symbol=val, api_key=api_key)
#         return globals()["table_"f"{val}"].reset_index().drop(labels=["index"], axis=1)

# df_currencies = pd.read_csv("data/main_currencies.csv")
# for currency_name, val in enumerate((df_currencies["currency code"])):
#     print(f'currency_name_{val}')

In [50]:
extract()

,Date,1. open,2. high,3. low,4. close,to
0,2022-09-30,1.01690,1.01840,1.01680,1.01780,EUR
1,2022-09-29,1.02690,1.03730,1.01780,1.01830,EUR
2,2022-09-28,1.04150,1.04810,1.02530,1.02700,EUR
3,2022-09-27,1.04010,1.04450,1.03420,1.04190,EUR
4,2022-09-26,1.03260,1.04660,1.02980,1.04050,EUR
...,...,...,...,...,...,...
13187,2014-11-28,1.17290,1.17820,1.17060,1.17500,AUD
13188,2014-11-27,1.17050,1.17460,1.16040,1.17320,AUD
13189,2014-11-26,1.17180,1.17900,1.16700,1.17090,AUD
13190,2014-11-25,1.16060,1.17430,1.16020,1.17180,AUD


In [15]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Float # https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_creating_table.htm
from sqlalchemy.engine import URL
from sqlalchemy.dialects import postgresql
from sqlalchemy.schema import CreateTable 

In [46]:

# create connection to database 
connection_url = URL.create(
    drivername = "postgresql+pg8000", 
    username = db_user,
    password = db_password,
    host = db_server_name, 
    port = 5432,
    database = db_database_name, 
)
engine = create_engine(connection_url)


In [47]:
def load(
    df:pd.DataFrame,
    load_method:str="overwrite",
    target_database_engine=engine,
    target_table_name:str=None
    )->None:
    if load_method.lower() == "overwrite": 
        df.to_sql(target_table_name, target_database_engine)

In [48]:
load(df=extract(), target_table_name='usd_exchange_rates')

KeyboardInterrupt: 